# Fil Rouge Part II - R on RNASeq and microarray data
## RNASeq part

Nom1_Nom2

---
---

## Synopsis du fil rouge RNASeq:

« Transcriptome par RNASeq du CMH dans les trois lignées lymphoblastoïdes PGF, COX et QBL porteuses à l’état homozygote d’haplotypes du CMH associés à des maladies autoimmunes »

Ce projet sous R fait suite au projet Fil Rouge RNASeq démarré sous Unix.
L’objectif principal sera d’effectuer une analyse de l’expression différentielle entre les trois lignées lymphoblastoïdes PGF, COX et QBL.

Pour rappel, ces trois lignées sont des lymphocytes B immortalisés par le virus EBV. Elles ont été établies à partir du sang de sujets présentant la particularité génétique d’être entièrement homozygotes pour la région du CMH et porteurs d’haplotypes associés à des maladies autoimmunes et infectieuses :

 >   - PGF avec HLA-A3-B7-Cw7-DR15 à risque vis-à-vis de la sclérose en plaques et du lupus érythémateux disséminé et protecteur vis à vis du diabète de type 1
 >   - COX  avec HLA-A1-B8-Cw7-DR3 à risque vis à vis du diabète de type 1, du lupus érythémateux disséminé et de la myasthénie acquise généralisée, des déficits immunitaires variables et de maladies infectieuses
 >   - QBL avec HLA-A26-B18-Cw5-DR3-DQ2 à risque pour le diabète de type 1 et la maladie de Basedow

L’ADN génomique de ces trois lignées a été entièrement séquencé par le MHC Haplotype project (https://www.ucl.ac.uk/cancer/research/department-cancer-biology/medical-genomics-group/past-projects/mhc-haplotype-project). La séquence de PGF est désormais la séquence officielle de référence du génome humain pour le CMH, tandis que les séquences des haplotypes alternatifs de COX et QBL sont également disponibles auprès du Genome Reference Consortium https://www.ncbi.nlm.nih.gov/grc/human).

Afin de mieux comprendre comment les différentes séquences haplotypiques peuvent influencer l’expression des gènes du CMH, vous comparerez cette expression dans les échantillons d’ARN de ces cellules préparées en deux réplicats biologiques de culture et quantifiée par RNASeq. Les six librairies ainsi séquencées sont nommées `GUP1` et `GUP2` pour les réplicats biologiques 1 et 2 de PGF, `GUP3` et `GUP4` pour les réplicats biologiques 1 et 2 de COX, et enfin `GUP5` et `GUP6` pour les réplicats biologiques de QBL. Pendant le TP Unix, vous avez travaillé sur le 1er réplicat de PGF et de COX. Vous allez désormais travailler avec les fichiers générés pour les 6 échantillons selon le même workflow sous Unix.

#### Rappel sur les étapes du worflow Unix :
    
__1. Données brutes.__ Vous avez vu un workflow d’analyse partant des données brutes de séquençage avec les reads au format `.fastq` contenant la séquence de chaque read/lecture avec un score de qualité pour chaque base du read. Il est possible d’effectuer un contrôle de qualité des fichiers .fastq par exemple avec l’outil __FASTQC__. Ici, les données étaient de bonne qualité et vous avez enchainé directement sur l’étape suivante.
    
__2. Nettoyage et filtrage des données.__ Vous avez utilisé l’outil __TRIMMOMATIC__ (http://www.usadellab.org/cms/?page=trimmomatic) sur les fichiers .fastq générant de nouveau fichiers `.fastq`. TRIMMOMATIC permet en effet d’éliminer les reads de mauvaise qualité : trop fort contenu en N (base non attribuée) ou score de qualité moyen de chaque base trop faible. De plus, lors de la génération des banques d’ARN à séquencer à partir d’un échantillon, les ARNs sont retro-transcrits en cDNA qui sont insérés entre deux adaptateurs lesquels permettront l’hybridation et l’amplification en phase solide directement sur la chambre de séquençage, avant le séquençage final des inserts. En principe, la taille des inserts est contrôlée. Cependant, il arrive que certains cDNA soient plus courts qu’attendus ou qu’il y ait des biais au moment de la ligation des adaptateurs, aboutissant dans la banque à des inserts plus petits que la longueur séquencée. La conséquence est que le séquençage continue au-delà de l’insert, et que les adaptateurs sont alors séquencés bien que sans intérêt. TRIMMOMATIC vise notamment à élaguer (« trimmer ») ces adapateurs en 3’ des reads. Enfin, les reads ici étaient appariés ou pairés car les deux extrémités des inserts étaient séquencés. TRIMMOMATIC s’assure que les reads conservés restent pairés et dans le même ordre dans chaque fichier.
    
__3. Cartographie des reads sur le génome de référence.__ Pour faciliter cette étape longue, de nombreux algorithmes ont été développés. L’algorithme de cartographie que vous avez utilisé est __HISAT2__ (https://ccb.jhu.edu/software/hisat2/index.shtml) qui procède en deux temps _(Pour des raisons pratiques, il est possible que vous ayiez finalement utilisé l'algoritme __STAR__ qui fonctionne de façon équivalente)_. La première étape dite d’indexation consiste à pré-processer le génome personnalisé à partir de sa séquence au format `.fasta`. L’objectif est de découper cette séquence en k-mers et de les trier afin de retrouver plus vite les alignements (un peu comme un dictionnaire classe les mots par ordre alphabétique). La seconde étape réalisée par HISAT2 est celle de la cartographie des reads du fichier `.fastq` sur le génome préprocessé générant ainsi des fichiers `.bam/.sam`. Ces fichiers contiennent les positions génomiques de chaque read cartographié. La structure d’un fichier sam/bam contient les alignements de chaque read. Il y a une ligne par alignement, contenant en particulier le nom de l’insert, la position génomique du read et celle de son ‘mate’ lorsqu’il s’agit de reads pairés, une valeur appelée FLAG indiquant notamment si le read est le premier de la paire ou le second, un score de qualité de la cartographie, la qualité de l’appariement (CIGAR string), la séquence du read et le score de qualité de chaque base, et des informations supplémentaires relative à l’alignement : unique, avec ou sans mismatch. Un même read peut avoir plusieurs alignements. Vous avez ensuite utilisé l’outil __SAMTools__ (http://www.htslib.org/) pour manipuler les fichiers bam obtenus et notamment compter le nombre de reads cartographiés avec ou sans mismatch. Vous avez pu les visualiser au moyen du browser __IGV__ (http://software.broadinstitute.org/software/igv/) avec la séquence de référence génomique au format `.fasta`. Ce browser vous a également permis de les superposer aux annotations des gènes fournies dans des fichiers `.gtf`. Les fichiers d’annotation fournis contenaient les coordonnées des exons, avec donc plusieurs lignes possibles pour un gène. 

*__Attention__* même si vous avez pu les visualiser, en aucun cas les fichiers `sam/bam` n’indiquent à quel gène correspondent les positions génomiques ! Dans le cadre du RNASeq, il fallait donc encore attribuer les reads aux "features" d'intérêt, ici les gènes afin de quantifier leur expression. 
__Deux étapes supplémentaires, les étapes 4 et 5, ont donc été réalisées sous Unix, par mes soins avant la partie R.__

__4. Comptage des reads par feature/gène.__ Cette étape a été réalisée sur les fichiers ̀.bam̀ avec les fichiers d’annotation `.gtf` afin de compter le nombre de reads correspondant à chaque gène au moyen de l’outil __featureCount__ du module __SUBREAD__ (http://subread.sourceforge.net/ ).
Cette étape a généré deux fichiers outputs par input :

`.counts` : avec le compte des reads par ENSEMBL geneID, c’est-à-dire les données « brutes » d’expression de chaque gène pour chaque échantillon

`.counts.summary` : un résumé des comptes de reads assignés pour le fichier d’annotation

La commande utilisée par featureCount est insérée comme commentaire en début du `.counts`. Vous pouvez essayer de comprendre cette commande en consultant la page: http://bioinf.wehi.edu.au/featureCounts/. L'outil featureCount a calculé les comptes par gène avec le paramètre "-g" dans la commande en regroupant tous les reads correspondant à chaque exon d'un même gène. Du coup, vous avez tous les intervalles exoniques qui sont affichés à la fois pour la colonne du chromosome, mais aussi pour les colonnes des positions de début, de fin, et de brin!
Les fichiers .counts sont disponibles sur adenine dans le répertoire `/srv/data/meg-m1-ue5/fil_rouge_r/rnaseq` :
> GUP-1_6p_on_chr6p-pgf_s1.counts
> GUP-2_6p_on_chr6p-pgf_s1.counts
> GUP-3_6p_on_chr6p-cox_s1.counts
> GUP-4_6p_on_chr6p-cox_s1.counts
> GUP-5_6p_on_chr6p-qbl_s1.counts
> GUP-6_6p_on_chr6p-qbl_s1.counts
	
__5. Extraction des informations pertinentes dans les fichiers `.gtf`.__ Les fichiers `.gtf` étant lourds à traiter à ce stade de votre apprentissage de la bioinfo, j’ai ajouté cette étape pour extraire les informations minimales pertinentes pour la suite du projet : gene_id, gene_name et gene_type du 9ème champ de chacun des fichiers `.gtf`. Cette étape peut être réalisée de différentes façons (R, awk…) mais un outil spécialement dédié à l’extraction de variables depuis un fichier `.gtf` appelé __tools.parsers.GTFTools__ est inclus  dans le packet pyhton __PyGeno__ (http://pygeno.iric.ca/). C’est celui-ci qui a été installé et utilisé, suivi de la commande « uniq » de bash. Si vous êtes interessés, les commandes successives pour COX sont fournies dans le fichier `Parse_GTF.txt` disponible dans le même repertoire. Les 3 fichiers d’information des gènes générés contiennent chacun 3 colonnes sans en-tête correspondant respectivement au « gene_id », « gene_name » et « gene_type » et sont disponibles sur  adenine dans le répertoire `/srv/data/meg-m1-ue5/fil_rouge_r/rnaseq`:
> pgf_gene_info.txt
> cox_gene_info.txt
> qbl_gene_info.txt

__=> En conclusion, sur les nouveaux fichiers que vous avez :__
> - Les fichiers `.counts` contiennent donc les comptes de reads cartographiés (comptes présents dans la dernière colonne), pour chacun des gènes (un par ligne, chaque gène ayant plusieurs exons...donc plusieurs positions indiquées dans les colonnes chromosome, début, fin et brin, séparées par des « ; ») avec son identifiant selon ENSEMBL en 1ère colonne.
__*ATTENTION*__  L’identifiant ENSEMBL d’un même gène diffère d’un haplotype à l’autre! Et certains gènes ou pseudogènes n’existent pas sur les trois haplotypes.
> - Les fichiers `info.text` contiennent la correspondance entre l’identifiant et le nom (ou symbole) de chaque gène. Notez que dans la colonne length, vous avez la longueur cumulée des intervalles, donc des exons, propre à l’haplotype considéré.


L’analyse d’expression différentielle de données de RNASeq est un sujet actif de recherche. Il existe actuellement un grand nombre de méthodes statistiques dédiées à ce type d’analyse sans qu’aucune méthode « gold standard » ne fasse encore référence. Ce fil rouge n’a bien sûr pas vocation à vous faire étudier ces différentes méthodes statistiques1 mais plutôt de vous proposer une initiation par première exploration de ces données de manière à la fois basique et intuitive en utilisant les fonctions « core » du langage R, tel que vous pourrez être amenés à le faire dans tout projet de bioinformatique.


---
---

### Section 1. Formattage des données

Les premières colonnes des fichiers `.counts` donnent pour chaque gène son identifiant selon ENSEMBL sur la séquence personnalisée suivi d’informations génomiques. La dernière colonne donne le compte de reads cartographiés pour le gène. Attention, certains gènes n’existent que sur un seul haplotype et l’identifiant ENSEMBL ID est propre à chaque séquence de référence : par exemple, le gène TNF a pour identifiant ENSG00000232810.3 sur 6p-pgf, ENSG00000204490.3  sur 6p-cox et ENSG00000206439.5 sur 6p-qbl !

__Etape 1 : faire un dataframe par lignée cellulaire__

    • Importez les 6 fichiers `.counts` et les 3 fichiers d’annotations `.txt` dans R.
    
    • Pour chaque lignée créez un dataframe incluant les informations d’annotation du fichier .txt, les informations communes aux fichiers .counts et les comptes de reads par gène pour chaque réplicat.
    
    • Organisez et nommez vos colonnes de façon adéquate.
    
    • Ne conservez que les lignes dont les noms de gènes donnés par « gene_name » sont présents une seule fois dans le dataframe. Vous pouvez utiliser les fonctions `table()` ou `duplicated()` pour les identifier. Attention la fonction `unique()` n’est pas adaptée dans ce cas car elle retourne toutes les valeurs distinctes une seule fois et non les valeurs n’ayant qu’une seule occurrence !

*__fonction recommandées :__*

`read.table()`

`str()`, `dim()`, `head()`, `tail()`

`names()`

`merge()` que vous pouvez éventuellement utiliser après avoir comparé les longueurs avec la fonction `length()`des vecteurs recensant les éléments uniques, partagés ou spécifiques entre deux vecteurs avec les fonctions `unique()`, `setdiff()`, `intersect()`. Par exemple `length(intersect(vecteur1, vecteur2))` vous retourne le nombre d’éléments en commun entre les vecteurs 1 et 2. En revanche,  `length(setdiff(vecteur1, vecteur2))` vous retourne le nombre d’éléments spécifique du vecteur 1.

`table()`, `duplicated()`

`which()`

>__Conseils/Explications supplémentaires pour cette étape 1 :__
>
>Le format des fichiers `.counts` importé dans R risque de vous surprendre. Lisez bien les informations dans la conclusion du synopsis de ce document sur le contenu des fichiers `.counts̀ et `info.text`. Pour vous aider, pour le `.counts` de l’échantillon GUP1, le data.frame obtenu à l’importation dans R doit contenir 379 lignes et seulement 7 colonnes, mais les colonnes 2 à 5 contiennent des chaînes de caractères avec des « ; » séparant les informations relatives à chaque exon !
Pour continuer avec l’exemple de PGF, vous devez donc créer un dataframe dont les premières colonnes viennent du fichier `info.txt`, les colonnes suivantes viennent toutes de `.counts` de GUP1, et enfin la dernière colonne de ̀.counts` de GUP2.  Vous pouvez utiliser la fonction `merge()` en spécifiant avec l'option `by` (ou by.x et by.y) le nom de la colonne, c'est à dire la colonne/variable commune entre les 2 dataframes que vous mergez = c'est la "clé". Dans le cours ma clé était un index (ou ID) mais c'était un exemple. Vous n'avez pas besoin de créer un index (c'est bien pour garder une trace de l'ordre initial), il faut surtout trouver la colonne commune qui sert de clé pour merger!)
>
>__Conseils plus généraux:__
>
>• Préférez l’utilisation directe des dataframes en utilisant les index (ou slicing) pour récupérer les colonnes, lignes ou valeurs, que de générer des vecteurs ou des des matrices intermédiaires. Je prendrai en compte dans l’évaluation la manipulation directe des dataframes.
>
>• Pour rappel, utilisez strinsgAsFactors=F lorsque vous lisez des fichiers dans R via read.table(). En empêchant la factorisation des données catégoriques, vous éviterez beaucoup de problèmes par la suite. Si vous avez malgré tout créé des facteurs, je vous renoivoie au tutoriel sur la manipulation des facteurs dans R de la session 2.

---
__Etape 2 : fusionner les  dataframes en un seul__

    • Générez un seul dataframe que vous appellerez `GUP` en combinant les comptes de reads des 6 échantillons pour les gènes communs uniquement. Pour ces gènes, vous conserverez le nom du gène, ses coordonnées (début et fin) sur la séquence de PGF uniquement, son type par haplotype et sa longueur par haplotype. Vous devez aboutir à un dataframe GUP de 279 lignes et 15 variables.

*__fonction recommandées :__*

`names()`

`merge()`que vous pouvez éventuellement utiliser après avoir comparé les longueurs avec la fonction length()des vecteurs recensant les éléments uniques, partagés ou spécifiques entre deux vecteurs avec les fonctions `unique()`, `setdiff()`, `intersect()`

`str()`, `dim()`, `head()`, `tail()`


>__Conseils/Explications supplémentaires pour cette étape 2 :__
>
> La fonction `merge()` ne vous permet de merger que deux dataframes à la fois.
> Mergez par exemple d’abord PGF à COX puis le résultat à QBL. 